<a href="https://colab.research.google.com/github/AmirKage/TDS-GroupProject24-25/blob/Group_work_AmirKage/Testing_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install Required Libraries
# Ensure you have all necessary libraries installed.

!pip install transformers datasets torch evaluate pandas scikit-learn




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 823.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
# Step 2: Load and Fix the Dataset
# We ensure that short_answer is always inside long_answer.

import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# Load the expanded dataset
df = pd.read_csv("Q&ADataset4.csv")

# Split dataset into 80% training and 20% evaluation
# Changed 'expanded_qa_df' to 'df'
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

# Save train and evaluation datasets separately
train_df.to_csv("qa_train.csv", index=False)
eval_df.to_csv("qa_eval.csv", index=False)

# Load the split datasets into HuggingFace format
dataset = load_dataset('csv', data_files={'train': 'qa_train.csv', 'eval': 'qa_eval.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [ ]:
from google.colab import userdata
userdata.get('Token_for_deepset/roberta-base-squad2')

In [ ]:
# Step 3: Load DistilBERT and Tokenizer
# We use DistilBERT because it's faster and lightweight.

# from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering

# Load tokenizer
#tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Load model
# model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")


from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")
tokenizer = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 4: Tokenize Data and Add Start/End Positions
# We map character positions to token positions.

def add_token_positions(example):
    # Tokenize the question and context
    encoding = tokenizer(
        example["question"],
        example["context"],  # Make sure this is the long passage
        truncation=True,
        padding="max_length",
        return_offsets_mapping=True
    )

    start_positions = []
    end_positions = []

    for i, (answer, context) in enumerate(zip(example["answer"], example["context"])):
        start_char = context.find(answer)

        # Handle cases where the answer isn't found
        if start_char == -1:
            start_positions.append(0)
            end_positions.append(0)
            continue

        end_char = start_char + len(answer)

        # Convert character positions to token positions
        offsets = encoding["offset_mapping"][i]

        token_start_index = None
        token_end_index = None

        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end:  # Find first token that includes the start_char
                token_start_index = idx
            if start < end_char <= end:  # Find last token that includes the end_char
                token_end_index = idx

        # If token positions are not found, default them to the first token
        if token_start_index is None:
            token_start_index = 0
        if token_end_index is None:
            token_end_index = token_start_index  # Avoid misalignment

        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    encoding["start_positions"] = start_positions
    encoding["end_positions"] = end_positions
    encoding.pop("offset_mapping")  # Remove offset mapping after use

    return encoding




from datasets import load_dataset

# Load dataset
dataset = load_dataset('csv', data_files={'train': 'qa_train.csv', 'eval': 'qa_eval.csv'})

# Apply preprocessing function with the fixed tokenization alignment
tokenized_dataset = dataset.map(add_token_positions, batched=True)



Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [ ]:
# Step 5: Fine-Tune DistilBERT
# We train DistilBERT on our dataset.

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,  # Keep the full 10 but apply early stopping
    weight_decay=0.01,
    learning_rate=2e-5,
    logging_steps=5,
    load_best_model_at_end=True,  # 🚀 Loads the best model at the lowest validation loss
    save_total_limit=2,  # 🚀 Keeps only the best 2 models to avoid overfitting
    metric_for_best_model="eval_loss",  # 🚀 Stops when eval loss stops improving
    greater_is_better=False,  # 🚀 Lower loss is better
)

# In previous attempts, The validation loss starts increasing after epoch 5, meaning the model might be overfitting.

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Step 6: Step 6: Fine-Tune DistilBERT
# We fine-tune the model using our expanded and properly split dataset.

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"],
)

# Train the model
trainer.train()

# Result: TrainOutput(global_step=125, training_loss=0.28027882409095767, metrics={'train_runtime': 7020.2375, 'train_samples_per_second': 0.142, 'train_steps_per_second': 0.018, 'total_flos': 261296756736000.0, 'train_loss': 0.28027882409095767, 'epoch': 5.0})

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: muhammadamir-shaikh (muhammadamir-shaikh-julius-maximus) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,4.458000,3.341571
2,1.529600,1.260177
3,0.538400,0.490845
4,0.097000,0.317880
5,0.122900,0.279977


TrainOutput(global_step=90, training_loss=1.614534391462803, metrics={'train_runtime': 3108.364, 'train_samples_per_second': 0.232, 'train_steps_per_second': 0.029, 'total_flos': 94070232023040.0, 'train_loss': 1.614534391462803, 'epoch': 5.0})

In [ ]:
# Step 7: Step 7: Evaluate the Model Using RoBERTa
import torch
import pandas as pd
import numpy as np
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

#EVALUATION

# Function to get model predictions
def get_predictions(model, tokenizer, dataset):
    predictions = []
    model.eval()

    for _, row in dataset.iterrows():
        inputs = tokenizer(
            row["question"],
            row["context"],  # Ensure "context" is the passage containing the answer
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        with torch.no_grad():
            outputs = model(**inputs)

        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        start_index = torch.argmax(start_logits, dim=1).item()
        end_index = torch.argmax(end_logits, dim=1).item()

        # Decode the predicted answer
        pred_answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1])
        predictions.append(pred_answer)

    return predictions

# Get predictions
eval_df["predicted_answer"] = get_predictions(model, tokenizer, eval_df)

# Function to compute Exact Match (EM) and F1-score
def compute_metrics(eval_df):
    exact_match = 0
    f1_scores = []

    for _, row in eval_df.iterrows():
        pred = row["predicted_answer"].strip().lower()
        actual = row["answer"].strip().lower()

        # Exact Match
        if pred == actual:
            exact_match += 1

        # Compute F1-score
        pred_tokens = set(pred.split())
        actual_tokens = set(actual.split())

        common_tokens = pred_tokens.intersection(actual_tokens)
        if len(common_tokens) == 0:
            f1 = 0
        else:
            precision = len(common_tokens) / len(pred_tokens)
            recall = len(common_tokens) / len(actual_tokens)
            f1 = 2 * (precision * recall) / (precision + recall)

        f1_scores.append(f1)

    # Compute final metrics
    exact_match_score = exact_match / len(eval_df) * 100
    f1_score = np.mean(f1_scores) * 100

    return {"Exact Match": exact_match_score, "F1 Score": f1_score}

# Compute evaluation results
results = compute_metrics(eval_df)
print("Evaluation Results:", results)

# Output: Evaluation Results: {'Exact Match': 6.0, 'F1 Score': 9.738095238095237}


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [ ]:
# Let's Finetune and train the Model again

import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer

# Load a stronger pre-trained model
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"  # Better than DistilBERT
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Load dataset
dataset = load_dataset('csv', data_files={'train': 'qa_train.csv', 'eval': 'qa_eval.csv'})

# ✅ Ensure token positions are aligned properly
def add_token_positions(example):
    encoding = tokenizer(
        example["question"],
        example["context"],  # Ensure "context" is used, not "answer"
        truncation=True,
        padding="max_length",
        return_offsets_mapping=True
    )

    start_positions = []
    end_positions = []

    for i, (answer, context) in enumerate(zip(example["answer"], example["context"])):
        start_char = context.find(answer)

        if start_char == -1:
            start_positions.append(0)
            end_positions.append(0)
            continue

        end_char = start_char + len(answer)

        offsets = encoding["offset_mapping"][i]
        token_start_index = None
        token_end_index = None

        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                token_start_index = idx
            if start < end_char <= end:
                token_end_index = idx

        if token_start_index is None:
            token_start_index = 0
        if token_end_index is None:
            token_end_index = token_start_index  # Prevent misalignment

        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    encoding["start_positions"] = start_positions
    encoding["end_positions"] = end_positions
    encoding.pop("offset_mapping")

    return encoding

# Apply preprocessing
tokenized_dataset = dataset.map(add_token_positions, batched=True)

# ✅ Fine-Tuning Hyperparameters
training_args = TrainingArguments(
    output_dir="./fine_tuned_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,  # Reduce if memory issue
    per_device_eval_batch_size=4,
    num_train_epochs=10,  # Increase to ensure proper learning
    weight_decay=0.01,
    learning_rate=3e-5,  # Small learning rate for fine-tuning
    logging_steps=5,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"],
)

# Train the model
trainer.train()

# ✅ Save the fine-tuned model
model.save_pretrained("./fine_tuned_results")
tokenizer.save_pretrained("./fine_tuned_results")

print("Fine-tuning complete! Model saved to './fine_tuned_results'")


Device set to use cpu


Exact Match: 5.56
F1 Score: 41.17


In [ ]:
# Evaluating the fine tuned model again

import torch
import pandas as pd
import numpy as np
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# ✅ Load the fine-tuned model and tokenizer
model_path = "./fine_tuned_results"  # Fine-tuned model directory
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()

# ✅ Load the evaluation dataset
eval_df = pd.read_csv("qa_eval.csv")

# ✅ Reduce dataset size for faster evaluation (optional)
eval_df = eval_df.head(100)  # Comment this line to evaluate the full dataset

# Function to get model predictions
def get_predictions(model, tokenizer, dataset):
    predictions = []

    for _, row in dataset.iterrows():
        inputs = tokenizer(
            row["question"],
            row["context"],  # Ensure "context" is used, not "answer"
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        with torch.no_grad():
            outputs = model(**inputs)

        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        start_index = torch.argmax(start_logits, dim=1).item()
        end_index = torch.argmax(end_logits, dim=1).item()

        # Decode the predicted answer
        pred_answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index + 1])
        predictions.append(pred_answer)

    return predictions

# ✅ Get predictions from the fine-tuned model
eval_df["predicted_answer"] = get_predictions(model, tokenizer, eval_df)

# Function to compute Exact Match (EM) and F1-score
def compute_metrics(eval_df):
    exact_match = 0
    f1_scores = []

    for _, row in eval_df.iterrows():
        pred = row["predicted_answer"].strip().lower()
        actual = row["answer"].strip().lower()

        # Exact Match
        if pred == actual:
            exact_match += 1

        # Compute F1-score
        pred_tokens = set(pred.split())
        actual_tokens = set(actual.split())

        common_tokens = pred_tokens.intersection(actual_tokens)
        if len(common_tokens) == 0:
            f1 = 0
        else:
            precision = len(common_tokens) / len(pred_tokens)
            recall = len(common_tokens) / len(actual_tokens)
            f1 = 2 * (precision * recall) / (precision + recall)

        f1_scores.append(f1)

    # Compute final metrics
    exact_match_score = exact_match / len(eval_df) * 100
    f1_score = np.mean(f1_scores) * 100

    return {"Exact Match": exact_match_score, "F1 Score": f1_score}

# ✅ Compute evaluation results
results = compute_metrics(eval_df)
print("🚀 Evaluation Results:", results)


Testing BERT-Large Model for training to improve exact match and F1 score

In [ ]:
# Step 1: install libraries
# Step 2: Load the dataset

#Step 3: Load BERT-Large Model and Tokenizer

# We replace DistilBERT with BERT-Large for better extraction performance.
from transformers import BertTokenizerFast, BertForQuestionAnswering

# Load tokenizer for BERT-Large
tokenizer = BertTokenizerFast.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# Load BERT-Large model
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Step 4: Tokenization & Fixing Answer Positions

def add_token_positions(example):
    encoding = tokenizer(
        example["question"], example["answer"], truncation=True, padding="max_length", return_offsets_mapping=True
    )

    start_positions = []
    end_positions = []

    for i, (answer, context) in enumerate(zip(example["answer"], example["answer"])):
        start_char = context.find(answer)

        # Handle cases where the answer isn't found
        if start_char == -1:
            start_positions.append(0)
            end_positions.append(0)
            continue

        end_char = start_char + len(answer)

        # Convert character positions to token positions
        offsets = encoding["offset_mapping"][i]

        token_start_index = next((idx for idx, (start, end) in enumerate(offsets) if start >= start_char), None)
        token_end_index = next((idx for idx, (start, end) in enumerate(offsets) if end <= end_char), None)

        # Ensure valid positions
        if token_start_index is None:
            token_start_index = 0
        if token_end_index is None:
            token_end_index = token_start_index  # Set end position to start position if misaligned

        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    encoding["start_positions"] = start_positions
    encoding["end_positions"] = end_positions
    encoding.pop("offset_mapping")  # Remove offset mapping after use

    return encoding

# Apply preprocessing
tokenized_dataset = dataset.map(add_token_positions, batched=True)


Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [ ]:
# Step 5: Configure Training Arguments
# We adjust learning rate and batch sizes to avoid overfitting.

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,  # 🚀 Reduce epochs to prevent overfitting
    learning_rate=2e-5,  # ✅ Lower learning rate for better stability
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=5,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Step 6: Train BERT-Large
# We train BERT-Large on our expanded dataset

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"],
)

# Train the model
trainer.train()


WHEN TESTING THIS MODEL, THE SYSTEM CRASHED AND RAM SPACE RAN OUT

Testing BERT-Large Model for training to improve exact match and F1 score
ALBERT: A lighter version of BERT that reduces memory consumption through parameter sharing and factorization techniques.

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import json
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import AlbertTokenizer, AlbertForQuestionAnswering, Trainer, TrainingArguments
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, pipeline
from evaluate import load

def load_dataset(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Load dataset
dataset_path = "albert_qa_dataset.json"
data = load_dataset(dataset_path)

# Split dataset into training (80%) and testing (20%)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Initialize tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
model = AlbertForQuestionAnswering.from_pretrained("albert-base-v2")

def encode_data(example):
    encoding = tokenizer(
        example["context"],
        example["question"],
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    start_idx = example["context"].find(example["answer"])
    end_idx = start_idx + len(example["answer"])
    encoding["start_positions"] = torch.tensor([max(0, start_idx)])
    encoding["end_positions"] = torch.tensor([max(0, end_idx)])
    return {key: val.squeeze(0) for key, val in encoding.items()}

class QADataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.encodings = [encode_data(ex) for ex in data]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.encodings[idx]

train_dataset = QADataset(train_data)
test_dataset = QADataset(test_data)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,4.833768
2,No log,3.961980
3,No log,3.811089


TrainOutput(global_step=3, training_loss=4.352688471476237, metrics={'train_runtime': 90.4708, 'train_samples_per_second': 0.133, 'train_steps_per_second': 0.033, 'total_flos': 265005637632.0, 'train_loss': 4.352688471476237, 'epoch': 3.0})

In [ ]:
# Save trained model
model.save_pretrained("./trained_albert")
tokenizer.save_pretrained("./trained_albert")

# Load different model for evaluation (e.g., DistilBERT)
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, pipeline

eval_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
eval_model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

evaluator = pipeline("question-answering", model=eval_model, tokenizer=eval_tokenizer)

# Evaluate on test dataset
for example in test_data:
    result = evaluator(question=example["question"], context=example["context"])
    print(f"Question: {example['question']}")
    print(f"Predicted Answer: {result['answer']}")
    print(f"Actual Answer: {example['answer']}")
    print("---")


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Question: Are you willing to allow us to process your personal data?
Predicted Answer: and using it for personalized services.
Actual Answer: No, I do not give consent for my data to be processed.
---


In [ ]:
!pip install datasets --upgrade  # Upgrade to the latest version of datasets if you haven't already.
!pip install evaluate

In [ ]:

import json
import random
import torch
import evaluate
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

# Step 1: Create a unique Q&A dataset
qa_data = [
    {"context": "The Eiffel Tower is located in Paris, France.", "question": "Where is the Eiffel Tower?", "answer": "Paris, France"},
    {"context": "Python is a programming language that is widely used for web development, data science, and automation.", "question": "What is Python used for?", "answer": "Web development, data science, and automation"},
    {"context": "The Amazon Rainforest is the largest rainforest in the world, covering millions of square kilometers.", "question": "What is the largest rainforest in the world?", "answer": "Amazon Rainforest"},
    {"context": "The moon orbits around the Earth approximately every 27.3 days.", "question": "How long does it take for the moon to orbit the Earth?", "answer": "27.3 days"},
    {"context": "Albert Einstein was awarded the Nobel Prize in Physics in 1921 for his contributions to theoretical physics.", "question": "When did Albert Einstein receive the Nobel Prize?", "answer": "1921"},
    {"context": "The Great Wall of China was built to protect Chinese states from invasions.", "question": "Why was the Great Wall of China built?", "answer": "To protect Chinese states from invasions"},
    {"context": "Water boils at 100 degrees Celsius at standard atmospheric pressure.", "question": "At what temperature does water boil?", "answer": "100 degrees Celsius"},
    {"context": "Mount Everest is the highest mountain in the world, standing at 8,848 meters.", "question": "What is the highest mountain in the world?", "answer": "Mount Everest"},
    {"context": "The human heart has four chambers: two atria and two ventricles.", "question": "How many chambers does the human heart have?", "answer": "Four chambers"},
    {"context": "The capital of Japan is Tokyo.", "question": "What is the capital of Japan?", "answer": "Tokyo"},
    {"context": "Photosynthesis is the process by which plants convert sunlight into energy.", "question": "What is photosynthesis?", "answer": "The process by which plants convert sunlight into energy"},
    {"context": "The Pacific Ocean is the largest ocean on Earth.", "question": "What is the largest ocean on Earth?", "answer": "Pacific Ocean"},
    {"context": "The human brain has approximately 86 billion neurons.", "question": "How many neurons are in the human brain?", "answer": "Approximately 86 billion neurons"},
    {"context": "The Statue of Liberty was a gift from France to the United States in 1886.", "question": "Who gifted the Statue of Liberty to the United States?", "answer": "France"},
    {"context": "Shakespeare wrote the play 'Romeo and Juliet'.", "question": "Who wrote 'Romeo and Juliet'?", "answer": "Shakespeare"},
    {"context": "The speed of light is approximately 299,792 kilometers per second.", "question": "What is the speed of light?", "answer": "Approximately 299,792 kilometers per second"},
    {"context": "The Great Pyramid of Giza is one of the Seven Wonders of the Ancient World.", "question": "Which ancient wonder is located in Egypt?", "answer": "The Great Pyramid of Giza"},
    {"context": "The Earth revolves around the Sun in about 365.25 days.", "question": "How long does it take for the Earth to revolve around the Sun?", "answer": "About 365.25 days"},
    {"context": "The smallest planet in our solar system is Mercury.", "question": "Which is the smallest planet in the solar system?", "answer": "Mercury"},
    {"context": "The chemical symbol for gold is Au.", "question": "What is the chemical symbol for gold?", "answer": "Au"},
    {"context": "H2O is the chemical formula for water.", "question": "What is the chemical formula for water?", "answer": "H2O"},
    {"context": "The human body has 206 bones.", "question": "How many bones are in the human body?", "answer": "206"},
    {"context": "The Wright brothers are credited with inventing the first successful airplane.", "question": "Who invented the first successful airplane?", "answer": "The Wright brothers"},
    {"context": "The Sahara Desert is the largest hot desert in the world.", "question": "What is the largest hot desert in the world?", "answer": "Sahara Desert"},
    {"context": "The United States declared independence in 1776.", "question": "When did the United States declare independence?", "answer": "1776"}
]

# Step 2: Split dataset (80% train, 20% test)
random.shuffle(qa_data)
split_idx = int(len(qa_data) * 0.8)
train_data = qa_data[:split_idx]
test_data = qa_data[split_idx:]

# Step 3: Convert to Hugging Face Dataset format
def preprocess_data(data):
    return {"context": [item["context"] for item in data],
            "question": [item["question"] for item in data],
            "answers": [{"text": [item["answer"]], "answer_start": [item["context"].find(item["answer"])]} for item in data]}

dataset = DatasetDict({
    "train": Dataset.from_dict(preprocess_data(train_data)),
    "test": Dataset.from_dict(preprocess_data(test_data))
})

# Step 4: Load model and tokenizer
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Step 5: Tokenization
def tokenize_fn(examples):
    # Tokenize question and context
    tokenized_inputs = tokenizer(
        examples["question"],
        examples["context"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True,  # Get offset mapping for answer span
    )

    # Find the start and end positions of the answer in the tokenized context
    offset_mapping = tokenized_inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for i, offsets in enumerate(offset_mapping):
        # Get answer span
        answer = examples["answers"][i]["text"][0]
        start_char = examples["answers"][i]["answer_start"][0]
        end_char = start_char + len(answer)

        # Find the start and end token positions
        sequence_ids = tokenized_inputs.sequence_ids(i)

        # Find token start index
        idx = 0
        while sequence_ids[idx] != 1:  # 1 corresponds to context
            idx += 1
        token_start_index = idx
        while sequence_ids[idx] == 1:
            if offsets[idx][0] <= start_char and offsets[idx][1] >= start_char:
                token_start_index = idx
            idx += 1

        # Find token end index
        token_end_index = idx - 1  # Set to last context token initially
        while sequence_ids[idx] == 1:
            if offsets[idx][0] <= end_char and offsets[idx][1] >= end_char:
                token_end_index = idx
            idx += 1

        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    # Add start and end positions to tokenized inputs
    tokenized_inputs["start_positions"] = start_positions
    tokenized_inputs["end_positions"] = end_positions

    return tokenized_inputs

# Step 6: Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)





In [ ]:
# Step 7: Train Model
# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_fn, batched=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"], # Use tokenized dataset
    eval_dataset=tokenized_datasets["test"]    # Use tokenized dataset
)

trainer.train()

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.508184
2,No log,0.412057
3,No log,0.499297


TrainOutput(global_step=30, training_loss=0.9923566182454427, metrics={'train_runtime': 542.1773, 'train_samples_per_second': 0.111, 'train_steps_per_second': 0.055, 'total_flos': 15677805404160.0, 'train_loss': 0.9923566182454427, 'epoch': 3.0})

In [ ]:
# Step 8: Evaluate Model
import evaluate
# metric = load_metric("squad")  # Replace this line with the following:
metric = evaluate.load("squad")  # Load the metric using evaluate.load

def evaluate():
    predictions = trainer.predict(tokenized_datasets["test"])

    # Get predicted start and end logits
    start_logits, end_logits = predictions.predictions

    # Get predicted answer spans using argmax
    predicted_answers = []
    for i in range(len(start_logits)):
        start_index = torch.argmax(torch.tensor(start_logits[i])).item()
        end_index = torch.argmax(torch.tensor(end_logits[i])).item()

        # Decode the answer span to text
        answer = tokenizer.decode(tokenized_datasets["test"][i]["input_ids"][start_index:end_index+1], skip_special_tokens=True)
        predicted_answers.append(answer)

    # Prepare predictions and references for metric computation
    preds = [{"id": str(i), "prediction_text": predicted_answers[i]} for i in range(len(predicted_answers))]
    references = [{"id": str(i), "answers": tokenized_datasets["test"]["answers"][i]} for i in range(len(test_data))]
    results = metric.compute(predictions=preds, references=references)
    print("Evaluation Results:", results)

evaluate()

Evaluation Results: {'exact_match': 20.0, 'f1': 48.757020757020754}


Testing the same deepset/roberta-base-squad2 model with a bigger dataset

In [ ]:
!pip install datasets --upgrade  # Upgrade to the latest version of datasets if you haven't already.
!pip install evaluate

In [ ]:
from google.colab import userdata
userdata.get('Token_for_deepset/roberta-base-squad2')

In [ ]:
import json
import torch
import random
import numpy as np
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import default_data_collator
import evaluate

In [ ]:
# Load dataset
with open("Q&ADataset4.json", "r", encoding="utf-8") as file:
    dataset = json.load(file)

In [ ]:
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

# Preprocess dataset
def preprocess_data(examples):
    inputs = tokenizer(
        examples["question"], examples["context"],
        truncation=True, padding="max_length",
        max_length=384, return_tensors="pt"
    )

    answer = examples["answer"]
    start_char = examples["context"].find(answer)
    end_char = start_char + len(answer)

    tokenized_inputs = tokenizer(
        examples["context"], return_offsets_mapping=True,
        max_length=384, truncation=True
    )

    offsets = tokenized_inputs["offset_mapping"]
    start_positions = end_positions = None

    for i, (start, end) in enumerate(offsets):
        if start <= start_char < end:
            start_positions = i
        if start < end_char <= end:
            end_positions = i

    if start_positions is None or end_positions is None:
        return None  # Skip this example if answer is out of range

    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "attention_mask": inputs["attention_mask"].squeeze(),
        "start_positions": torch.tensor(start_positions, dtype=torch.long),
        "end_positions": torch.tensor(end_positions, dtype=torch.long)
    }



In [ ]:
# Convert dataset to processed format
data = [preprocess_data(d) for d in dataset]
data = [d for d in data if d is not None]  # Remove None values

# Split dataset (80% train, 20% test)
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_data, test_data = random_split(data, [train_size, test_size])

class QADataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_dataset = QADataset(train_data)
test_dataset = QADataset(test_data)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,  # Increase batch size
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    learning_rate=3e-5,  # Adjusted learning rate
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",  # Save checkpoints after each epoch
    # metric_for_best_model="f1",  # Track best model using F1-score
)





trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate model
# metric = load_metric("squad_v2")
# metric = evaluate.load("squad")

metric = evaluate.load("squad_v2")


def compute_metrics(predictions, references):
    return metric.compute(predictions=predictions, references=references)

results = trainer.evaluate()
print("Evaluation Results:", results)

In [ ]:
# Compute exact match and F1 score
#squad_metric = load_metric("squad_v2")



squad_metric = evaluate.load("squad_v2")

predictions = trainer.predict(test_dataset).predictions
start_logits, end_logits = predictions

def postprocess_predictions(start_logits, end_logits, dataset):
    predictions = []
    for i in range(len(dataset)):
        input_ids = dataset[i]["input_ids"]
        start_idx = torch.argmax(torch.tensor(start_logits[i])).item()
        end_idx = torch.argmax(torch.tensor(end_logits[i])).item()

        # Ensure valid indices
        if start_idx >= end_idx:
            continue  # Skip bad predictions

        predicted_answer = tokenizer.decode(input_ids[start_idx:end_idx+1], skip_special_tokens=True)

        # Ensure predictions have sufficient length
        if len(predicted_answer.split()) < 2:
            continue  # Skip very short predictions

        predictions.append({
            "id": str(i),
            "prediction_text": predicted_answer
        })

    return predictions


formatted_predictions = postprocess_predictions(start_logits, end_logits, test_dataset)
references = [{"id": str(i), "answers": {"text": [dataset[i]["answer"]], "answer_start": [dataset[i]["context"].find(dataset[i]["answer"])]}} for i in range(len(test_dataset))]

squad_results = squad_metric.compute(predictions=formatted_predictions, references=references)
print("Exact Match:", squad_results["exact"])  # Changed key to "exact"
print("F1 Score:", squad_results["f1"])